# Analyse Géo-Spatiale : À la recherche du terrain de football parfait dans le Val-d'Oise

## Contexte et Objectifs

Ce notebook a pour ambition de transformer un jeu de données brutes en une carte interactive et parlante. En nous appuyant sur le **Recensement des Équipements Sportifs (RES)**, disponible en open data sur le portail de la région Île-de-France (source : [data.iledefrance.fr](http://data.iledefrance.fr/explore/dataset/recensement_des_equipements_sportifs_dans_le_val-d_oise/download?format=csv)), nous allons cartographier l'ensemble des infrastructures où la pratique du football est possible dans le département du Val-d'Oise (95). L'objectif est de créer un outil visuel, simple et direct, permettant à quiconque – un joueur amateur, un club local, un parent ou même un urbaniste – d'identifier rapidement les terrains disponibles à proximité.

Au-delà de la simple localisation, cette analyse est une démonstration de la puissance de la data science appliquée au bien commun. En quelques lignes de code, nous allons filtrer, structurer et visualiser une information d'intérêt public pour la rendre accessible et utile. Ce projet illustre comment des données ouvertes peuvent être mobilisées pour répondre à des questions très concrètes : où se trouve le terrain couvert le plus proche pour s'entraîner l'hiver ? Quels sont les types de surface (synthétique, gazon naturel) disponibles dans ma commune ? Ce notebook sert de guide pas-à-pas pour passer de la donnée à la décision.

## Exploration du jeu de données

### Description fonctionnelle du fichier
Ce jeu de données est un inventaire exhaustif des installations et des équipements sportifs localisés dans le département du Val-d'Oise (95). Il offre une vision à la fois macroscopique (par installation, comme un complexe sportif) et microscopique (par équipement, comme un court de tennis spécifique au sein du complexe). La richesse des informations permet d'analyser le parc sportif sous de multiples angles : géographique, technique, accessibilité, vétusté, et usages. C'est un outil stratégique pour l'aménagement du territoire, l'évaluation des politiques publiques sportives et l'analyse des inégalités territoriales en matière d'accès au sport.

### Champs notables
- **Hiérarchie des données** : Les colonnes préfixées par `inst_` décrivent l'installation (le "contenant", ex: Gymnase Léo Lagrange), tandis que celles préfixées par `equip_` décrivent un équipement spécifique au sein de cette installation (le "contenu", ex: le terrain de basket). La granularité est donc à l'équipement.
- **Géolocalisation** : Les colonnes `longitude` et `latitude` sont fondamentales pour toute analyse spatiale. Elles permettent de cartographier précisément chaque équipement. La colonne `coordonnees` est une concaténation, mais il est préférable d'utiliser les champs dédiés.
- **Accessibilité** : Une série de champs booléens (`equip_pmr_acc`, `equip_pmr_sanit`, etc.) et le champ textuel `accessibilite_aux_personnes_a_mobilite_reduite` fournissent des informations cruciales sur l'adaptation des infrastructures aux personnes en situation de handicap. C'est une mine d'or pour les études sur l'inclusion.
- **Caractéristiques techniques** : Des colonnes comme `equip_type_name`, `equip_sol`, `equip_nature` (Intérieur/Découvert), `equip_surf`, `equip_long`, `equip_larg` permettent de qualifier très finement chaque équipement.
- **Vétusté et maintenance** : `equip_service_date`, `equip_service_periode`, et `equip_travaux_date` sont des indicateurs temporels précieux pour évaluer l'âge du parc, anticiper les besoins de rénovation et analyser les vagues d'investissement passées.
- **Contexte socio-territorial** : La présence des champs `qpv` (Quartier Prioritaire de la politique de la Ville) et `dens_lib` (niveau de densité de la commune) permet de croiser l'offre sportive avec des indicateurs de politique de la ville et de typologie de territoire.

## Méthodologie

Notre démarche se décompose en trois étapes clés, formant un pipeline de traitement de données classique et efficace :

1.  **Chargement et Requêtage des Données** : Après avoir chargé le jeu de données complet, nous utilisons la puissance du SQL pour interroger directement le dataframe en mémoire. Nous rédigeons une requête précise pour filtrer uniquement les équipements du Val-d'Oise (`dep_code` = '95'), liés à la pratique du football (`activites` LIKE '%Football%'), et disposant de coordonnées géographiques valides.
2.  **Préparation des Données** : Le résultat de la requête est un jeu de données propre et ciblé, contenant toutes les informations nécessaires pour notre cartographie : nom de l'installation, de l'équipement, nature (couvert/découvert), type de sol et coordonnées.
3.  **Visualisation Cartographique** : À l'aide de la bibliothèque Folium, nous projetons ces points de données sur une carte interactive. Chaque terrain est représenté par un marqueur cliquable, dont la couleur varie s'il est couvert ou en plein air. Une fenêtre contextuelle (popup) révèle les détails de chaque équipement, offrant une lecture riche et immédiate de l'information.

## Mise en œuvre de l'analyse

### Étape 1 : Extraction des données pertinentes via SQL

La première étape consiste à sculpter notre jeu de données. Plutôt que de manipuler l'ensemble du fichier, nous formulons une requête SQL qui agit comme un scalpel pour extraire avec précision la substantifique moelle : les terrains de football du 95. La clause `ILIKE '%Football%'` garantit que nous capturons toutes les variations (ex: "Football", "Football à 7", etc.), tandis que les filtres sur les coordonnées assurent la qualité des données pour la cartographie.

```sql
SELECT
    "inst_nom" AS nom_installation,
    "equip_nom" AS nom_equipement,
    "equip_nature" AS nature_equipement,
    "equip_sol" AS type_sol,
    "activites",
    CAST("latitude" AS DOUBLE) AS latitude,
    CAST("longitude" AS DOUBLE) AS longitude
FROM
    loaded_dataset
WHERE
    "dep_code" = '95'
    AND "activites" ILIKE '%Football%'
    AND "latitude" IS NOT NULL
    AND "longitude" IS NOT NULL
ORDER BY
    nom_installation
```

Le résultat est un dataframe `pandas` parfaitement formaté, prêt à être visualisé.

### Étape 2 : Création de la carte interactive

C'est ici que la magie opère. Nous transformons notre tableau de données en une carte vivante. Le code ci-dessous initialise une carte centrée sur le Val-d'Oise, puis parcourt chaque ligne de notre dataframe pour y placer un marqueur. Pour rendre la carte plus intuitive, nous utilisons des icônes de football et un code couleur simple : vert pour les terrains découverts et orange pour les terrains intérieurs. Chaque marqueur, au survol, affiche le nom de l'installation, et au clic, révèle une fiche détaillée de l'équipement.

```python
import pandas as pd
import duckdb as ddb
import folium
from folium.features import CustomIcon
import pandas as pd

# En supposant que le dataframe 'df' est le résultat de la requête SQL précédente

# Filtrer les équipements de football (redondant si déjà fait en SQL, mais bonne pratique)
mask = df['activites'].str.contains('Football', case=False, na=False)
g = df[mask].copy()

# Trier par nom d'installation pour une meilleure lisibilité des popups
g = g.sort_values(['nom_installation', 'nom_equipement'])

# Coordonnées du centre pour la vue initiale
center_coords = [g['latitude'].mean(), g['longitude'].mean()]

# Créer la carte
dataviz = folium.Map(
    location=center_coords,
    zoom_start=10,
    tiles='CartoDB positron'
)

# Fonction pour définir des icônes personnalisées
def get_icon(nature):
    # Couleurs par nature d'équipement
    color_map = {
        'Découvert': 'green',
        'Intérieur': 'orange',
    }
    color = color_map.get(nature, 'gray')
    return folium.Icon(icon='futbol', prefix='fa', color=color)

# Ajouter les marqueurs à la carte
for _, row in g.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(
            html=f"""
            <div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; font-size: 14px;">
                <h4 style="margin: 5px 0;">{row['nom_installation']}</h4>
                <p style="margin: 3px 0;"><strong>Équipement :</strong> {row['nom_equipement']}</p>
                <p style="margin: 3px 0;"><strong>Nature :</strong> {row['nature_equipement']}</p>
                <p style="margin: 3px 0;"><strong>Surface :</strong> {row['type_sol']}</p>
                <p style="margin: 3px 0;"><strong>Activités :</strong> {row['activites']}</p>
            </div>
            """,
            max_width=300
        ),
        tooltip=row['nom_installation'],
        icon=get_icon(row['nature_equipement'])
    ).add_to(dataviz)

# Ajouter une légende pour une meilleure compréhension
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 180px; height: 90px; 
     background-color: white; z-index: 1000; font-size: 14px;
     border:2px solid grey; border-radius: 5px;
     padding: 10px;
     font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
     <p style="margin: 5px;"><i class="fa fa-futbol" style="color:green;"></i> Terrains Découverts</p>
     <p style="margin: 5px;"><i class="fa fa-futbol" style="color:orange;"></i> Terrains Intérieurs</p>
     </div>
'''
dataviz.get_root().html.add_child(folium.Element(legend_html))

# Afficher la carte
# dataviz
```

## Pistes d'exploration et analyses complémentaires

Cette carte n'est qu'un point de départ. La richesse du jeu de données initial ouvre la voie à des analyses bien plus poussées qui pourraient inspirer des politiques publiques ou des stratégies de développement.

1.  **Analyse de la "fracture sportive"** : Corréler la densité d'équipements par type (`equip_type_famille`) avec la présence dans un `qpv`. On pourrait ainsi identifier les quartiers sous-dotés en certains types d'infrastructures (piscines, dojos, etc.).
2.  **Modélisation prédictive de la vétusté** : Utiliser `equip_service_date` et `equip_travaux_date` comme variables pour construire un modèle qui prédit les besoins de rénovation. On pourrait enrichir le modèle avec `equip_utilisateur` (un équipement très utilisé par les scolaires s'use différemment) pour affiner les prédictions.
3.  **Analyse de l'accessibilité réelle** : Au-delà du booléen `inst_acc_handi_bool`, on peut créer un score d'accessibilité composite en agrégeant les multiples champs `equip_pmr_*` pour chaque installation. La cartographie de ce score révélerait les zones les mieux et les moins bien adaptées.
4.  **Clustering des communes sportives** : Appliquer un algorithme de clustering (type K-Means) sur les communes en se basant sur la diversité et la quantité de leurs équipements. Cela permettrait de dégager des profils types de communes ("la commune du tennis", "la commune des sports collectifs", "le pôle aquatique", etc.).
5.  **Analyse d'adéquation offre/demande potentielle** : En croisant ces données avec des données démographiques de l'INSEE (pyramide des âges, revenus), on pourrait évaluer si l'offre d'équipements est en phase avec le profil des habitants d'un territoire donné. Par exemple, y a-t-il assez de city-stades dans les zones à forte population jeune ?

## Mots clés
#SportPourTous #OpenData #DataAnalyse #ValDOise #Infrastructures #AménagementTerritoire #PolitiquesPubliques📈 #Accessibilité♿️ #Cartographie🗺️ #Géomarketing

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("http://data.iledefrance.fr/explore/dataset/recensement_des_equipements_sportifs_dans_le_val-d_oise/download?format=csv", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT
    "inst_nom" AS nom_installation,
    "equip_nom" AS nom_equipement,
    "equip_nature" AS nature_equipement,
    "equip_sol" AS type_sol,
    "activites",
    CAST("latitude" AS DOUBLE) AS latitude,
    CAST("longitude" AS DOUBLE) AS longitude
FROM
    loaded_dataset
WHERE
    "dep_code" = '95'
    AND "activites" ILIKE '%Football%'
    AND "latitude" IS NOT NULL
    AND "longitude" IS NOT NULL
ORDER BY
    nom_installation """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

Cette datavisualisation utilise la bibliothèque Folium pour créer une carte interactive, un choix idéal pour représenter des données géographiques. L'utilisation de marqueurs personnalisés et de couleurs distinctes permet de localiser rapidement les terrains de football et d'identifier s'ils sont couverts ou en plein air, rendant l'information dense accessible et intuitive.

In [3]:
import pandas as pd
import duckdb as ddb
import folium
from folium.features import CustomIcon
import pandas as pd

# Filtrer les équipements de football
mask = df['activites'].str.contains('Football', case=False, na=False)
g = df[mask].copy()

# Trier par nom d'installation pour une meilleure lisibilité des popups
g = g.sort_values(['nom_installation', 'nom_equipement'])

# Coordonnées du centre
center_coords = [g['latitude'].mean(), g['longitude'].mean()]

# Créer la carte
dataviz = folium.Map(
    location=center_coords,
    zoom_start=11,
    tiles='CartoDB positron'
)

# Définir des icônes personnalisées par type d'équipement
def get_icon(nature):
    # Couleurs par nature d'équipement
    color_map = {
        'Découvert': 'green',
        'Intérieur': 'orange',
    }
    color = color_map.get(nature, 'gray')
    return folium.Icon(icon='futbol', prefix='fa', color=color)

# Ajouter les marqueurs
for _, row in g.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(
            html=f"""
            <div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; font-size: 14px;">
                <h4 style="margin: 5px 0;">{row['nom_installation']}</h4>
                <p style="margin: 3px 0;"><strong>Équipement :</strong> {row['nom_equipement']}</p>
                <p style="margin: 3px 0;"><strong>Nature :</strong> {row['nature_equipement']}</p>
                <p style="margin: 3px 0;"><strong>Surface :</strong> {row['type_sol']}</p>
                <p style="margin: 3px 0;"><strong>Activités :</strong> {row['activites']}</p>
            </div>
            """,
            max_width=300
        ),
        tooltip=row['nom_installation'],
        icon=get_icon(row['nature_equipement'])
    ).add_to(dataviz)

# Ajouter une légende
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 180px; height: 90px; 
     background-color: white; z-index: 1000; font-size: 14px;
     border:2px solid grey; border-radius: 5px;
     padding: 10px;
     font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
     <p style="margin: 5px;"><i class="fa fa-futbol" style="color:green;"></i> Découvert</p>
     <p style="margin: 5px;"><i class="fa fa-futbol" style="color:orange;"></i> Intérieur</p>
     </div>
'''
dataviz.get_root().html.add_child(folium.Element(legend_html))
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*